# Question Answer ChatBot

## 1) Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import keras

Using TensorFlow backend.


## 2) Reading data from file

In [0]:
#For training

with open('drive/My Drive/Pytorch_DataSet/TextFiles/train_qa.txt','rb') as f:
  train_data = pickle.load(f)

In [0]:
#For testing

with open('drive/My Drive/Pytorch_DataSet/TextFiles/test_qa.txt','rb') as f:
  test_data = pickle.load(f)

In [6]:
len(train_data), len(test_data)

(10000, 1000)

In [8]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [13]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [14]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [16]:
''.join(train_data[0][2])

'no'

In [17]:
type(train_data), type(test_data)

(list, list)